In [1]:
import pandas as pd
import numpy as np
import math

import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam

import numpy as np
import random
from collections import deque

import sys

/Users/xiongzhuoran/anaconda3/envs/python3.4/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [21]:
data_1 = pd.read_csv('Data_Daily_Stock_Dow_Jones_30/dow_jones_30_daily_price.csv')

In [62]:
equal_4711_list = list(data_1.tic.value_counts() == 4711)
names = data_1.tic.value_counts().index
select_stocks_list = list(names[equal_4711_list])+['NKE','KO']

In [65]:
data_2 = data_1[data_1.tic.isin(select_stocks_list)][~data_1.datadate.isin(['20010912','20010913'])]

/Users/xiongzhuoran/anaconda3/envs/python3.4/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [77]:
data_3 = data_2[['iid','datadate','tic','prccd','ajexdi']]

In [79]:
data_3['adjcp'] = data_3['prccd'] / data_3['ajexdi']

/Users/xiongzhuoran/anaconda3/envs/python3.4/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [92]:
daily_data = {}

for date in np.unique(data_3.datadate):
    daily_data[date] = data_3[data_3.datadate == date]

In [94]:
daily_data[20160513].adjcp

,iid,datadate,tic,prccd,ajexdi,adjcp
4116,1,20160513,AXP,64.12,1.0,64.12
8827,1,20160513,AAPL,90.52,1.0,90.52
13538,1,20160513,VZ,50.94,1.0,50.94
18249,1,20160513,BA,132.12,1.0,132.12
22960,1,20160513,CAT,70.07,1.0,70.07
27671,1,20160513,JPM,61.20,1.0,61.20
52558,1,20160513,CVX,100.74,1.0,100.74
57270,1,20160513,KO,45.35,1.0,45.35
61981,1,20160513,DIS,100.52,1.0,100.52
66692,1,20160513,DWDP,50.55,1.0,50.55


In [95]:
daily_data[20110513]

,iid,datadate,tic,prccd,ajexdi,adjcp
2858,1,20110513,AXP,49.49,1.0,49.490000
7569,1,20110513,AAPL,340.50,7.0,48.642857
12280,1,20110513,VZ,37.26,1.0,37.260000
16991,1,20110513,BA,79.03,1.0,79.030000
21702,1,20110513,CAT,106.33,1.0,106.330000
26413,1,20110513,JPM,43.15,1.0,43.150000
51300,1,20110513,CVX,102.39,1.0,102.390000
56012,1,20110513,KO,68.18,2.0,34.090000
60723,1,20110513,DIS,41.52,1.0,41.520000
65434,1,20110513,DWDP,38.37,1.0,38.370000


In [3]:
# prints formatted price
def formatPrice(n):
	return ("-$" if n < 0 else "$") + "{0:.2f}".format(abs(n))

In [4]:
# returns the vector containing stock data from a fixed file
def getStockDataVec(key):
    vec = []
    lines = open("data/" + key + ".csv", "r").read().splitlines()

    for line in lines[1:]:
        vec.append(float(line.split(",")[4]))

In [5]:
# returns the sigmoid
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [6]:
# returns an an n-day state representation ending at time t
def getState(data, t):
    block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
    res = []
    res.append(sigmoid(block[i + 1] - block[i]))
    
    return np.array([res])

In [7]:
# prints formatted price
def formatPrice(n):
    return ("-$" if n < 0 else "$") + "{0:.2f}".format(abs(n))

In [8]:
# returns the vector containing stock data from a fixed file
def getStockDataVec(key):
    vec = []
    lines = open("data/" + key + ".csv", "r").read().splitlines()

    for line in lines[1:]:
        vec.append(float(line.split(",")[4]))

    return vec

In [9]:
# returns the sigmoid
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [10]:
# returns an an n-day state representation ending at time t
# returns [p1...p28| m | share1 ... share28]
def getState(data, t, n):
    price_list = data[t]
    res = []
    for i in range(n - 1):
        res.append(sigmoid(block[i + 1] - block[i]))

    return np.array([res])

In [11]:
class Agent:
    def __init__(self, is_eval=False, model_name=""):
        
        self.state_size = 57     # [p1...p28| m | share1 ... share28]
        
        self.action_size = 28    # [share1 ... share28]    
        
        self.memory = deque(maxlen=1000)
        self.inventory = []
        self.model_name = model_name
        self.is_eval = is_eval

        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995

        self.model = load_model("models/" + model_name) if is_eval else self._model()

    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
        model.add(Dense(units=32, activation="relu"))
        model.add(Dense(self.action_size, activation="softmax"))
        model.compile(loss="mse", optimizer=Adam(lr=0.001))

        return model

    def act(self, state):
        if not self.is_eval and np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)

        options = self.model.predict(state)
        return np.argmax(options[0])

    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])

        for state, action, reward, next_state, done in mini_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay 

In [12]:
stock_name, episode_count = ['^GSPC', 1000]

agent = Agent()
data = daily_data
l = len(data) - 1
batch_size = 32

for e in range(episode_count + 1):
	print("Episode " + str(e) + "/" + str(episode_count))
	state = getState(data, 0, window_size + 1)

	total_profit = 0
	agent.inventory = []

	for t in range(l):
		action = agent.act(state)

		# sit
		next_state = getState(data, t + 1, window_size + 1)
		reward = 0

		if action == 1: # buy
			agent.inventory.append(data[t])
			print("Buy: " + formatPrice(data[t]))

		elif action == 2 and len(agent.inventory) > 0: # sell
			bought_price = agent.inventory.pop(0)
			reward = max(data[t] - bought_price, 0)
			total_profit += data[t] - bought_price
			print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

		done = True if t == l - 1 else False
		agent.memory.append((state, action, reward, next_state, done))
		state = next_state

		if done:
			print("--------------------------------")
			print("Total Profit: " + formatPrice(total_profit))
			print("--------------------------------")

		if len(agent.memory) > batch_size:
			agent.expReplay(batch_size)

	if e % 10 == 0:
		agent.model.save("models/model_ep" + str(e))

Episode 0/1000
Buy: $1318.55
Buy: $1326.65
Sell: $1347.97 | Profit: $29.42
Sell: $1342.54 | Profit: $15.89
Buy: $1364.30
Buy: $1357.51
Sell: $1354.95 | Profit: -$9.35
Buy: $1364.17
Buy: $1373.73
Sell: $1366.01 | Profit: $8.50
Buy: $1373.47
Sell: $1349.47 | Profit: -$14.70
Buy: $1340.89
Sell: $1332.53 | Profit: -$41.20
Sell: $1315.92 | Profit: -$57.55
Buy: $1326.61
Buy: $1301.53
Sell: $1278.94 | Profit: -$61.95
Sell: $1255.27 | Profit: -$71.34
Sell: $1252.82 | Profit: -$48.71
Buy: $1245.86
Buy: $1267.65
Sell: $1239.94 | Profit: -$5.92
Buy: $1241.23
Buy: $1234.18
Buy: $1241.41
Buy: $1253.80
Buy: $1264.74
Sell: $1233.42 | Profit: -$34.23
Sell: $1180.16 | Profit: -$61.07
Sell: $1197.66 | Profit: -$36.52
Sell: $1166.71 | Profit: -$74.70
Sell: $1173.56 | Profit: -$80.24
Buy: $1150.53
Sell: $1122.14 | Profit: -$142.60
Buy: $1117.58
Sell: $1139.83 | Profit: -$10.70
Sell: $1182.17 | Profit: $64.59
Buy: $1147.95
Sell: $1160.33 | Profit: $12.38
Buy: $1145.87
Sell: $1106.46 | Profit: -$39.41
Buy: 

KeyboardInterrupt: 

In [15]:
agent.model.save("models/model_ep" + str(e))

In [16]:
from keras.models import load_model

In [20]:
stock_name, model_name = ['^GSPC_2011','model_ep0']
model = load_model("models/" + model_name)
window_size = model.layers[0].input.shape.as_list()[1]

agent = Agent( True, model_name)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32

state = getState(data, 0, window_size + 1)
total_profit = 0
agent.inventory = []

for t in range(l):
	action = agent.act(state)

	# sit
	next_state = getState(data, t + 1, window_size + 1)
	reward = 0

	if action == 1: # buy
		agent.inventory.append(data[t])
		print("Buy: " + formatPrice(data[t]))

	elif action == 2 and len(agent.inventory) > 0: # sell
		bought_price = agent.inventory.pop(0)
		reward = max(data[t] - bought_price, 0)
		total_profit += data[t] - bought_price
		print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

	done = True if t == l - 1 else False
	agent.memory.append((state, action, reward, next_state, done))
	state = next_state

	if done:
		print("--------------------------------")
		print(stock_name + " Total Profit: " + formatPrice(total_profit))
		print("--------------------------------")

Buy: $1347.32
Buy: $1335.10
Sell: $1340.20 | Profit: -$7.12
Sell: $1346.29 | Profit: $11.19
Buy: $1260.34
Buy: $1200.07
Buy: $1199.38
Sell: $1119.46 | Profit: -$140.88
Sell: $1172.53 | Profit: -$27.54
Sell: $1120.76 | Profit: -$78.62
Buy: $1165.24
Sell: $1198.62 | Profit: $33.38
Buy: $1166.76
Sell: $1129.56 | Profit: -$37.20
Buy: $1195.19
Sell: $1246.96 | Profit: $51.77
--------------------------------
^GSPC_2011 Total Profit: -$195.02
--------------------------------
